In [32]:
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional

from NLP_little_helpers import *
little_helpers()

potions: re, numpy as np, pandas as pd, pickle, json, nltk, keras, collections
spells: clean_data, predict_tags, tagged_n_grams, unknown_words_X, check_and_predict


In [3]:
with open('quotes-100-en.json', encoding="utf8") as file:
    dct_text = json.load(file)

In [28]:
text = clean_data([q for person, quotes in dct_text.items() for q in quotes])
print("Total of {} quotes.".format(len(text)))

Total of 41538 quotes.


In [24]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print("Total of {} unique words.".format(vocab_size))
 
# separate into input(X) and output(y)
sequences = np.array(pad_sequences(sequences))
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

max_length = X[0].shape[0]

In [37]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, max_length+1, input_length=seq_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 29, 30)            939750    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 29, 256)           162816    
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 31325)             1033725   
Total params: 2,220,547
Trainable params: 2,220,547
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=5, shuffle=True, validation_split=0.2)
 # save the model to file
model.save('sequencing_model.h5')

C:\Users\felip\Miniconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 33230 samples, validate on 8308 samples
Epoch 1/5
33152/33230 [============================>.] - ETA: 0s - loss: 4.6027e-10 - accuracy: 1.0000